In [1]:
import numpy as np

In [7]:
n22 = np.zeros((2,2))

In [8]:
a=dict()

In [9]:
a['cat']=n22

In [10]:
a

{'cat': array([[0., 0.],
        [0., 0.]])}

In [16]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


In [51]:
class Element:
    test_size = 0.3 # 30% data as the test case
    
    def __init__(self, name):
        self.name = name[0]
        self.data = self.name
        

In [52]:
ME=Element("me")

In [53]:
ME.data

'm'

In [1]:
import pandas as pd
import numpy as np

import glob
from ase.io import read
from dscribe.descriptors import ACSF

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics

from pandas import set_option
pd.options.mode.chained_assignment = None

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from scipy.stats import truncnorm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [36]:
class Structure:
    
    def __init__(self, pdb_path):
        self.name = pdb_path.split('/')[1].split('.pdb')[0]
        self.water = self.name.split('.')[2]
        self.system = self.name.split('.')[0]
        self.loop = self.name.split('.')[1]
        self.mdframe = self.name.split('.')[3]
        self.feat_atom=dict()
        print("PDB name is ", self.name)
        
    def calc_sym(self):
        '''
        The function takes the structure file and compute the 
        symmetry function for each atom
        output: a list of symmetry functions for each atom. 
                Length of the list = number of atoms
        ''' 
        pdb_file = self.name+'.pdb'
        chg_file = self.name + '.chg'
        structure = read("pdb/"+pdb_file)
        feat_lst=[]
        num=structure.get_global_number_of_atoms()
        atomPos = [x for x in range(0,num)]

        chg=np.genfromtxt("charge/"+chg_file)


        for r_cut in [x/2. for x in range(2,13,1)]:
            # Setting up the ACSF descriptor
            acsf = ACSF(
                species=["C", "H", "O", "N", "Ca"],
                rcut=r_cut,
                g2_params=[[0.0001,0]],
            )
            feat_lst.append(acsf.create(structure,positions=atomPos,n_jobs=4))

        acsf_ang = ACSF(
            species=["C", "H", "O", "N", "Ca"],
            rcut=3.0,
            g4_params=[[0.0001,0.5,-1]],
        )
        feat_lst.append(acsf_ang.create(structure,positions=atomPos,n_jobs=4))

        feat = np.hstack(tuple(feat_lst))
        dat = np.hstack((feat,chg))
        np.savetxt('sym/sym_'+pdb_file[0:-4]+'.dat', dat, delimiter=',')
        #self.sym_func = dat
        aN={1:'H',6:'C',7:'N',8:'O',20:'Ca'}
        for i in dat:
            ikey = aN[int(i[-2])]
            print(ikey)
            if ikey in self.feat_atom:
                self.feat_atom[ikey] = np.vstack((self.feat_atom[ikey],i))
            else:
                print(ikey,"is not in the dictionary, adding ...")
                self.feat_atom[ikey] = i

In [37]:
pdb1=Structure("pdb/apoCaM-CA-Ng.loop3.water4.frame574.pdb")

PDB name is  apoCaM-CA-Ng.loop3.water4.frame574


In [38]:
pdb1.calc_sym()

C
C is not in the dictionary, adding ...
C
O
O is not in the dictionary, adding ...
H
H is not in the dictionary, adding ...
H
H
N
N is not in the dictionary, adding ...
C
C
O
C
C
O
O
H
H
H
H
N
C
C
O
C
C
C
C
N
H
H
H
H
H
H
H
H
H
H
H
H
H
N
C
C
O
C
C
O
O
H
H
H
H
N
C
C
O
H
H
H
N
C
C
O
C
C
N
O
H
H
H
H
H
H
N
C
C
O
H
H
H
N
C
C
O
C
C
C
C
C
C
C
O
H
H
H
H
H
H
H
H
H
N
C
C
O
C
C
C
C
H
H
H
H
H
H
H
H
H
H
H
N
C
C
O
C
O
H
H
H
H
H
N
C
C
O
C
H
H
H
H
H
N
C
C
O
C
H
H
H
H
H
N
C
C
O
C
C
C
O
O
H
H
H
H
H
H
N
C
H
H
H
H
Ca
Ca is not in the dictionary, adding ...
O
H
H
O
H
H
O
H
H
O
H
H


In [31]:
water=int(pdb1.water[5:])

In [32]:
pdb

4

In [33]:
pdb1.feat_atom['Ca'].shape


KeyError: 'Ca'

In [97]:
feat_atom=dict()
aN={1:'H',6:'C',7:'N',8:'O',20:'CA'}
for i in pdb1.sym_func:
    feat_atom[aN[int(i[-2])]]=i


In [147]:
pdb1.feat_atom['H']

array([[0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.1481],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.1481],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.1481],
       ...,
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.365 ],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.365 ],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.365 ]])

In [154]:
import numpy as np
def my_custom_loss_func(y_true, y_pred):
    diff = np.abs(y_true - y_pred).min()
    return np.log1p(diff)

# score will negate the return value of my_custom_loss_func,
# which will be np.log(2), 0.693, given the values for X
# and y defined below.
from sklearn.metrics import fbeta_score, make_scorer
score = make_scorer(my_custom_loss_func, greater_is_better=False)
X = [[1], [1]]
y = [0, 1]

from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf = clf.fit(X, y)
my_custom_loss_func(y, clf.predict(X))
score(clf, X, y)

-0.6931471805599453

In [157]:
[x/4. for x in range(4,13,1)]

[1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0]

In [158]:
testsize = 0.3
rstate=random.randrange(1,999999999,1)

In [159]:
loops = glob.glob("pdb/*.pdb")
structures = dict()
counter = 0
count_total = len(loops)
for loop in loops:
    name = loop.split('/')[1].split('.pdb')[0]
    structures[name]=np.zeros((2,2))
    

In [176]:
X=np.array([i for i in structures.keys()])
y=np.array([i for i in structures.values()])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = testsize, random_state = rstate)

In [178]:
X_train.shape

(5471,)

In [1]:
feat_lst

NameError: name 'feat_lst' is not defined

In [34]:
a = [[1,2],[3,4],[5,6]]

In [46]:
b = np.array(a)

In [62]:
n = 1
nwater=None

In [63]:
b[0:nwater, 0:]

array([[1, 2],
       [3, 4],
       [5, 6]])

In [64]:
from ase.io import read
from dscribe.descriptors import CoulombMatrix
from dscribe.descriptors import ACSF

In [65]:
structure = read('pdb/holoCaM.loop3.water2.frame368.pdb')

In [71]:
structure.get_distances(1,1)

array([0.])

In [83]:
la=structure[178]

In [88]:
la.symbol

'H'

In [89]:
i=structure.get_global_number_of_atoms()
cutoff = 4
cutAtoms=dict()

for a_index in range(i):
    
    atom = structure[a_index]
    elem = atom.symbol
    
    if elem not in cutAtoms:
        cutAtoms[elem]=list()
    cutAtoms[elem].append(structure[structure.get_distances(a_index, range(i)) < cutoff])

In [129]:
maxi = 0
for i in cutAtoms['Ca']:
    i_n = i.get_global_number_of_atoms()
    if maxi < i_n:
        maxi = i_n
print(maxi)

25


In [138]:
cm = CoulombMatrix(n_atoms_max=26)

In [139]:
firstcarbon = cm.create(cutAtoms['Ca'][0])

In [163]:
firstcarbon.shape

(1, 676)

In [119]:
b.sum()

121

In [122]:
a=dict()
b=dict()

In [123]:
a['h']=3
a['i']=4

In [124]:
b['h']=5

In [127]:
{**a, **b}

{'h': 5, 'i': 4}

In [146]:
chg = np.genfromtxt("charge/holoCaM.loop3.water2.frame368.chg")

In [153]:
chg[0
   ]

array([ 6.    , -0.5989])

In [154]:
c={1:3,2:4}

In [166]:
cutAtoms['C'][0]

Atoms(symbols='C2OH3NCH3', pbc=False, atomtypes=..., bfactor=..., occupancy=..., residuenames=..., residuenumbers=...)

In [162]:
cm.create(cutAtoms['C'][0]).shape

(1, 676)

In [167]:
a

{'h': 3, 'i': 4}

In [168]:
b

{'h': 5}

In [2]:
a=np.array([1,3])

In [3]:
b=np.array([2,4])

In [4]:
c=np.vstack((a,b,b))

In [5]:
c

array([[1, 3],
       [2, 4],
       [2, 4]])

In [175]:
d = np.vstack([a,b,b])

In [176]:
d

array([[1, 3],
       [2, 4],
       [2, 4]])

In [7]:
c[:,:]

array([[1, 3],
       [2, 4],
       [2, 4]])